In [ ]:
# !pip install --quiet mlflow dagshub



# if CONFIG["is_submission"]:
!ls /kaggle/input/pyvips-python-and-deb-package
# intall the deb packages
!yes | dpkg -i --force-depends /kaggle/input/pyvips-python-and-deb-package/linux_packages/archives/*.deb
# install the python wrapper
!pip install --quiet pyvips -f /kaggle/input/pyvips-python-and-deb-package/python_packages/ --no-index

import pyvips



In [ ]:


import os
import gc
import cv2
import datetime
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt
from skimage import io


# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
"""from cancer_utils_tiles import EarlyStopping, fetch_scheduler
from cancer_utils_tiles import CancerTilesDataset, TilesInferenceDataset
from cancer_utils_tiles import print_logged_info, get_dataloaders, data_transforms
from cancer_utils_tiles import get_train_file_path, get_test_file_path, get_or_create_experiment_id"""


# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
# warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
pd.set_option('display.max_rows', 200)

In [ ]:
CONFIG = {
    "is_submission": False,
    "datetime_now": datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), 
    "n_fold": 5,
    "fold": 1,
    "seed": 42,
    "img_size": 512,
    "model_name": "tf_efficientnet_b0_ns",
    "num_classes": 5,
    "train_batch_size": 16,
    "valid_batch_size": 16,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "num_epochs": 30,
    "early_stopping": True,
    "patience": 7,
    "learning_rate": 1e-4,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-6,
    "T_max":15,
    "weight_decay": 1e-6,
    "anomaly_class":"MC",
    "n_tiles": 2,
}


In [ ]:
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TRAIN_DIR = "/kaggle/input/tiles-of-cancer-2048px-scale-0-25/"
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_images'


# 1. Get Data

In [ ]:
def delete_tiles(directory_path):
    if os.path.isdir(directory_path):
        for filename in os.listdir(directory_path):
            if os.path.isfile(os.path.join(directory_path, filename)):
                os.remove(os.path.join(directory_path, filename))
                
def pad_tile_repeat(tile, h,w):
    tile_ = tile
    tile_size = (h, w) if tile.ndim == 2 else (h, w, tile.shape[2])
    
    # Initialize a new tile with the desired size
    tile = np.zeros(tile_size, dtype=tile.dtype)
    
    # Determine the amount to pad on the bottom and right
    pad_y = h - tile_.shape[0]
    pad_x = w - tile_.shape[1]
    
    # Copy the original tile into the new tile
    tile[:tile_.shape[0], :tile_.shape[1], ...] = tile_

    # Expand on the right by repeating the left side
    if pad_x > 0:
        tile[:, -pad_x:] = tile[:, :pad_x]

    # Expand on the bottom by repeating the top side
    if pad_y > 0:
        tile[-pad_y:, :] = tile[:pad_y, :]
    return tile

def pad_tile_continue(tile, h,w):
    tile_ = tile
    pad_y = h - tile_.shape[0]
    pad_x = w - tile_.shape[1]
    # Pad the tile using the 'edge' mode, which continues the border
    tile = np.pad(tile_, ((0, pad_y), (0, pad_x), (0, 0)), mode='edge')
    return tile
                
def pad_tile_mirroring(tile, h,w):
    tile_ = tile
    tile_size = (h, w) if tile.ndim == 2 else (h, w, tile.shape[2])
    
    # Initialize a new tile with the desired size
    tile = np.zeros(tile_size, dtype=tile.dtype)
    
    # Determine the amount to pad on each side
    pad_y = h - tile_.shape[0]
    pad_x = w - tile_.shape[1]
    
    # Mirror the edges for padding
    # Vertical padding
    top = tile_[:pad_y, :] if pad_y > 0 else tile_
    bottom = tile_[-pad_y:, :] if pad_y > 0 else tile_
    vertical_padded = np.concatenate([top[::-1], tile_, bottom[::-1]], axis=0)
    
    # Horizontal padding
    left = vertical_padded[:, :pad_x] if pad_x > 0 else vertical_padded
    right = vertical_padded[:, -pad_x:] if pad_x > 0 else vertical_padded
    tile = np.concatenate([left[:, ::-1], vertical_padded, right[:, ::-1]], axis=1)
    return tile 


def extract_image_tiles(
    p_img, img_id, tmp_dir, size: int = 2048, scale: float = 0.5,
    drop_thr: float = 0.8, white_thr: int = 245, max_samples: int = 50
) -> list:
    # delete_tiles(tmp_dir)  # empty directory from previous images
    im = pyvips.Image.new_from_file(p_img)
    w = h = size
    # https://stackoverflow.com/a/47581978/4521646
    idxs = [(y, y + h, x, x + w) for y in range(0, im.height, h) for x in range(0, im.width, w)]
    # random subsample
    max_samples = max_samples if isinstance(max_samples, int) else int(len(idxs) * max_samples)
    random.seed(42)
    random.shuffle(idxs)
    images = []
    i = 0
    for y, y_, x, x_ in (idxs):
        i += 1
        img_path = f"{tmp_dir}/{img_id}_{str(i)}.png"
        # https://libvips.github.io/pyvips/vimage.html#pyvips.Image.crop
        tile = im.crop(x, y, min(w, im.width - x), min(h, im.height - y)).numpy()[..., :3]
        if (tile.shape[0] != h) & (tile.shape[1] != w):
            tile = im.crop(im.width - w, im.height-h, w, h).numpy()[..., :3]
        elif(tile.shape[0] != h):
            tile = im.crop(x, im.height-h, w, h).numpy()[..., :3]
        elif(tile.shape[1] != w):
            tile = im.crop(im.width - w, y, w, h).numpy()[..., :3]
            
            # tile = pad_tile_continue(tile, h,w)
            #tile_ = tile
            #tile_size = (h, w) if tile.ndim == 2 else (h, w, tile.shape[2])
            #tile = np.zeros(tile_size, dtype=tile.dtype)
            #tile[:tile_.shape[0], :tile_.shape[1], ...] = tile_
        black_bg = np.sum(tile, axis=2) == 0
        tile[black_bg, :] = 255
        mask_bg = np.mean(tile, axis=2) > white_thr
        #if np.sum(mask_bg) >= (np.prod(mask_bg.shape) * drop_thr):
            #print(f"skip almost empty tile: {k:06}_{int(x_ / w)}-{int(y_ / h)}")
        #    continue
        # print(tile.shape, tile.dtype, tile.min(), tile.max())
        new_size = int(size * scale), int(size * scale)
        tile = Image.fromarray(tile).resize(new_size, Image.LANCZOS)
        tile.save(img_path)
        images.append(img_path)
        # need to set counter check as some empty tiles could be skipped earlier
        if len(images) >= max_samples:
            break
    return images


class CreateTilesDataset(Dataset):

    def __init__(
        self,
        img_path: str,
        img_id: str = None,
        tmp_dir: str = None,
        size: int = 2048,
        scale: float = 0.25,
        white_thr: int = 225,
        thr_max_bg: float = 0.6,
        max_samples: int = 30,
        transforms = None,
        is_submission: bool = True,
    ):
        self.max_samples = max_samples
        self.white_thr = white_thr
        self.thr_max_bg = thr_max_bg
        self.is_submission = is_submission
        
        self.transforms = transforms
            # print(img_path)
        assert os.path.isfile(img_path)
        self.imgs = extract_image_tiles(
            img_path, img_id, tmp_dir, size=size, scale=scale,
            drop_thr=self.thr_max_bg, max_samples=max_samples)
        """    all_imgs = glob.glob(os.path.join(img_path, img_id, "*.png"))
            # Filter images based on background threshold
            self.imgs = []
            for img_path in all_imgs:
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                black_bg = np.sum(img, axis=2) == 0
                img[black_bg, :] = 255
                mask_bg = np.mean(img, axis=2) > self.white_thr
                if np.sum(mask_bg) <= (np.prod(mask_bg.shape) * self.thr_max_bg):
                    self.imgs.append(img_path)  # Include this image
            self.imgs = self.imgs[:self.max_samples]"""
        print(f"Number of save tiles: {len(self.imgs)}")
            # print(self.imgs)

    def __getitem__(self, idx: int) -> tuple:
        img = cv2.imread(self.imgs[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # filter background
        mask = np.sum(img, axis=2) == 0
        img[mask, :] = 255
        if np.max(img) < 1.5:
            img = np.clip(img * 255, 0, 255).astype(np.uint8)
        if self.transforms:
            img = self.transforms(image=img)["image"]
        return img

    def __len__(self) -> int:
        return len(self.imgs)

In [ ]:
df_train = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
df_train["is_tma"] = df_train["is_tma"].astype(bool)
df_tma = df_train[df_train["is_tma"]==True]
df_tma.shape

In [ ]:
import pyvips
from PIL import Image

In [ ]:
try:
    os.mkdir("UBC_TMA_tiles_1024px_scale05_v7")
except:
    pass
is_submission = True
for i, row in df_tma.iterrows():
    # row = dict(idx_row[1])
    img_id = str(row["image_id"])
    tmp_dir = "UBC_TMA_tiles_1024px_scale05_v7/"+img_id
    os.mkdir(tmp_dir)
    print("Image ID: ", img_id)
    print("Save to: ", tmp_dir)
    dataset = CreateTilesDataset(
        os.path.join("/kaggle/input/UBC-OCEAN/", "train_images", f"{img_id}.png"), img_id=img_id,
        size=1024, scale=0.5, max_samples=100,
    tmp_dir=tmp_dir)

In [ ]:
len(train_loader_tma.dataset)

In [ ]:
train_loader_tma, valid_loader_tma, df_train_fold = get_dataloaders(df_train_min_tiles.head(1).copy(), TRAIN_DIR, CONFIG, data_transforms_orig, n_tiles=420, train_val_split=1)
print(len(train_loader_tma.dataset))
train_images = next(iter(train_loader_tma))
for i in range(len(train_loader_tma.dataset)):
    print(i)
    print(train_images["image"][i].shape)
plot_tiles(train_loader_tma, num_samples=len(train_loader_tma.dataset))

In [ ]:
import os
files_dir = dict()
for dirname, _, filenames in os.walk("/kaggle/working/UBC_TMA_tiles_1024px_scale05_v2/4134"):
    print(dirname)
    i = 0
    for filename in filenames:
        print(f"{dirname}/{filename}")
        im = Image.open(f"{dirname}/{filename}")
        print(im.size)
        # print(os.path.join(dirname, filename))
        i+=1
    files_dir[dirname] = i
    print(f"Number of tiles: {i}")

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

def plot_images_from_folder(folder_path, N):
    # Get list of image file paths
    images = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('png', 'jpg', 'jpeg'))][:N]
    
    # Calculate grid size
    grid_size = int(N**0.5)
    if grid_size * grid_size < N:
        grid_size += 1

    # Create subplots
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
    axes = axes.flatten()

    # Plot each image
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(os.path.basename(img_path))

    # Turn off empty subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

# Example usage
# plot_images_from_folder('/path/to/your/folder', 9)


In [ ]:
path = "/kaggle/working/UBC_TMA_tiles_1024px_scale05_v7/48734"
plot_images_from_folder(path, 16)

In [ ]:
path = "/kaggle/working/UBC_TMA_tiles_1024px_scale05/91"
plot_images_from_folder(path, 9)